In [2]:
import numpy as np
import pandas as pd

counts_exons_raw = pd.read_csv("E:/Transcriptomics_V1/Mouse/Tasic/GSE115746_cells_exon_counts.csv")
counts_introns_raw = pd.read_csv("E:/Transcriptomics_V1/Mouse/Tasic/GSE115746_cells_intron_counts.csv")
metadata = pd.read_csv("E:/Transcriptomics_V1/Mouse/Tasic/GSE115746_complete_metadata_28706-cells.csv")

In [6]:
counts_introns_raw

,Unnamed: 0,F2S4_150422_002_A01,F2S4_150422_002_B01,F2S4_150422_002_C01,F2S4_150422_002_D01,F2S4_150422_002_E01,F2S4_150422_002_F01,F2S4_150422_002_G01,F2S4_150422_002_H01,F2S4_150427_001_A01,...,F1S4_180124_316_G01,F1S4_180124_316_H01,F1S4_180124_317_A01,F1S4_180124_317_B01,F1S4_180124_317_C01,F1S4_180124_317_D01,F1S4_180124_317_E01,F1S4_180124_317_F01,F1S4_180124_317_G01,F1S4_180124_317_H01
0,0610005C13Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0610006L08Rik,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0610007P14Rik,3,0,48,27,0,12,0,0,8,...,0,0,0,0,10,0,0,1,0,0
3,0610009B22Rik,0,0,0,0,1,0,0,0,0,...,0,0,2,0,1,0,0,0,0,0
4,0610009E02Rik,0,0,16,0,0,0,0,0,7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45763,n-R5s142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45764,n-R5s143,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45765,n-R5s144,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45766,n-R5s146,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
counts_exons = counts_exons_raw.copy()
counts_exons.index = np.array(counts_exons["Unnamed: 0"])
counts_exons = counts_exons[counts_exons.columns[1:]]

counts_introns = counts_introns_raw.copy()
counts_introns.index = np.array(counts_introns["Unnamed: 0"])
counts_introns = counts_introns[counts_introns.columns[1:]]

In [89]:
counts = counts_exons + counts_introns

In [90]:
counts.shape

(45768, 23178)

In [17]:
metadata.columns

Index(['sample_name', 'title', 'source_name', 'organism', 'donor_id',
       'donor_sex', 'donor_genotype', 'injection_type', 'injection_target',
       'injected_material', 'dissected_region', 'dissected_layer',
       'facs_gating', 'facs_date', 'rna_amplification_set', 'sequencing_tube',
       'sequencing_batch', 'sequencing_qc_pass_fail', 'cell_class',
       'cell_subclass', 'cell_cluster', 'molecule', 'SRA_Run', 'GEO_Sample',
       'GEO_Sample_Title'],
      dtype='object')

In [102]:
metadata_visp = metadata[metadata["dissected_region"] == "VISp"]

In [92]:
counts = counts.T

In [103]:
bcs = [bc for bc in list(metadata_visp[["sample_name"]].to_numpy().flatten()) if bc in counts.index]
gns = counts.columns.tolist()

In [106]:
metadata_visp.index = np.array(metadata_visp["sample_name"], dtype=str)
metadata_visp = metadata_visp[metadata_visp.columns[1:]]

In [107]:
metadata_visp = metadata_visp.loc[bcs]

In [123]:
metadata_visp[["cell_subclass"]] = metadata_visp[["cell_subclass"]].fillna("None")
metadata_visp[["cell_cluster"]] = metadata_visp[["cell_cluster"]].fillna("None")

In [113]:
from scipy.sparse import csc_matrix
from scipy.io import mmwrite
import gzip
import shutil

def compress_file(input_filename, output_filename):
    with open(input_filename, 'rb') as f_in:
        with gzip.open(output_filename, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

path = "E:/Transcriptomics_V1/Mouse/Tasic/filtered_feature_bc_matrix/"
X = counts.loc[bcs][gns]
mmwrite(path + "matrix.mtx", csc_matrix(X.values, dtype=np.int64).T, precision=1)
X.columns.to_frame().to_csv(path + "features.tsv", index=False, header=False)
X.index.to_frame().to_csv(path + "barcodes.tsv", index=False, header=False)
metadata_visp[["donor_id"]].to_csv(path + "sample.csv", index=False, header=False)
metadata_visp[["cell_class"]].to_csv(path + "class.csv", index=False, header=False)
metadata_visp[["cell_subclass"]].to_csv(path + "subclass.csv", index=False, header=False)
metadata_visp[["cell_cluster"]].to_csv(path + "type.csv", index=False, header=False)
for fl in ["matrix.mtx", "features.tsv", "barcodes.tsv"]:
    compress_file(path + fl, path + fl + ".gz")

In [131]:
metadata_visp[["donor_id"]].value_counts()

donor_id
254569      358
245824      335
254570      285
237585      264
227236      255
           ... 
293794        2
313296        1
280708        1
353938        1
341898        1
Name: count, Length: 199, dtype: int64